## Analyzing Tweets

We can gather a sample of Twitter data using the Twitter API (https://dev.twitter.com).  To do so, we'll need to create a Twitter application and get credentials for it.  You can do this manually at https://app.twitter.com.  Once you have an app, go to the "Key and Access Tokens" tab to find your credentials.

In [7]:
import random
import twitter
import emoji
import itertools
import pandas as pd
from itertools import chain
from keras.preprocessing.sequence import pad_sequences
import numpy as np
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import one_hot
import keras.callbacks
import json

import os
import nb_utils
from keras.layers import Input, Conv1D, MaxPooling1D, Flatten, Dense, Dropout, Merge, LSTM, Embedding, GlobalMaxPooling1D
from keras.models import Model
from keras.layers.merge import Concatenate, Average

from gensim.models import Word2Vec

In [2]:
# Fill these in!

CONSUMER_KEY = 'xbMuxcJpRTiVGt2C2EYnA'
CONSUMER_SECRET = '2DbQTsvIptkPTdaUcos8DDvQH9fzO0hNjJpUT2uVzQ'
ACCESS_TOKEN = '7319442-EDm4CPxL7W4KkZcGWRMJNVHp88W5OH9vgblu898fg'
ACCESS_SECRET = '5ZxJSbqXhG7uhgXzTFWf9XhkfsxxinlPRXyDTzbA9w'

In [3]:
auth=twitter.OAuth(
    consumer_key=CONSUMER_KEY,
    consumer_secret=CONSUMER_SECRET,
    token=ACCESS_TOKEN,
    token_secret=ACCESS_SECRET,
)

status_stream = twitter.TwitterStream(auth=auth).statuses

[x['text'] for x in itertools.islice(status_stream.sample(), 0, 5) if x.get('text')]

['RT @saru__0925: ほんとに人生１回でいいから踊ってみたい😩😩 https://t.co/dsM1jFgPpS']

In [4]:
status_stream = twitter.TwitterStream(auth=auth).statuses

def english_has_emoji(tweet):
    if tweet.get('lang') != 'en':
        return False
    return any(ch for ch in tweet.get('text', '') if ch in emoji.UNICODE_EMOJI)

%time tweets = list(itertools.islice(filter(english_has_emoji, status_stream.sample()), 0, 100))

CPU times: user 4.78 s, sys: 608 ms, total: 5.39 s
Wall time: 43.7 s


In [5]:
stripped = []
for tweet in tweets:
    text = tweet['text']
    emojis = {ch for ch in text if ch in emoji.UNICODE_EMOJI}
    if len(emojis) == 1:
        emoiji = emojis.pop()
        text = ''.join(ch for ch in text if ch != emoiji)
        stripped.append((text, emoiji))
len(stripped)

64

## Using the CNN

Let's see what the CNN of the previous chapter does on the data

In [3]:
all_tweets = pd.read_csv('data/emojis.csv')
all_tweets['emoji'].value_counts()

😂    124823
❤     43218
😍     40566
😭     35714
😊     20076
🙄     17963
😩     16232
🔥     15453
🤔     15419
💕     12026
💯     11783
😘     11065
💀      9928
✨      9886
🙃      9405
👀      7842
😒      7019
☺      6871
😢      6846
😳      6716
💙      6616
😎      6349
😉      6272
😅      6133
😁      6010
😌      5759
😏      5623
💖      5331
😔      5244
😴      4999
      ...  
🤤         1
🦍         1
🕡         1
↩         1
🏣         1
🔀         1
🤠         1
📭         1
👘         1
📇         1
📯         1
📐         1
🕣         1
📂         1
🕍         1
🎽         1
🈹         1
🔣         1
🏦         1
🏗         1
🥇         1
🚡         1
🛐         1
🏺         1
🎚         1
🤷         1
🔏         1
🎑         1
🤡         1
🏬         1
Name: emoji, Length: 989, dtype: int64

In [4]:
tweets = all_tweets.groupby('emoji').filter(lambda c:len(c) > 1000)
tweets['emoji'].value_counts()

😂    124823
❤     43218
😍     40566
😭     35714
😊     20076
🙄     17963
😩     16232
🔥     15453
🤔     15419
💕     12026
💯     11783
😘     11065
💀      9928
✨      9886
🙃      9405
👀      7842
😒      7019
☺      6871
😢      6846
😳      6716
💙      6616
😎      6349
😉      6272
😅      6133
😁      6010
😌      5759
😏      5623
💖      5331
😔      5244
😴      4999
      ...  
✌      1523
📸      1496
🎤      1487
🌚      1452
👅      1431
🏈      1373
🌟      1355
⏩      1332
❗      1325
🔴      1304
☕      1296
👊      1273
👇      1259
❣      1254
🎧      1246
🎈      1210
⏭      1198
💫      1181
↪      1157
🤑      1152
⚽      1141
😹      1112
😶      1108
💦      1075
😣      1074
😥      1072
🙁      1066
🤕      1065
😰      1013
☀      1013
Name: emoji, Length: 121, dtype: int64

In [5]:
max(tweets['text'], key=lambda t:len(t))

"Don't worry, my love, I don't get you wrong Don't get me wrong. We're connected with our hearts, souls, minds, bodies. If any doubt, we ask"

In [10]:
chars = list(sorted(set(chain(*tweets['text']))))
char_to_idx = {ch: idx for idx, ch in enumerate(chars)}
max_sequence_len = max(len(x) for x in tweets['text'])

emojis = list(sorted(set(tweets['emoji'])))
emoji_to_idx = {em: idx for idx, em in enumerate(emojis)}
emojis[:10]

train_tweets, test_tweets = train_test_split(tweets, test_size=0.1)


In [10]:
def data_generator(tweets, batch_size):
    while True:
        if batch_size is None:
            batch = tweets
            batch_size = batch.shape[0]
        else:
            batch = tweets.sample(batch_size)
        X = np.zeros((batch_size, max_sequence_len, len(chars)))
        y = np.zeros((batch_size,))
        for row_idx, (_, row) in enumerate(batch.iterrows()):
            y[row_idx] = emoji_to_idx[row['emoji']]
            for ch_idx, ch in enumerate(row['text']):
                X[row_idx, ch_idx, char_to_idx[ch]] = 1
        yield X, y

next(data_generator(tweets, 10))

(array([[[ 0.,  0.,  0., ...,  0.,  0.,  0.],
         [ 0.,  0.,  0., ...,  0.,  0.,  0.],
         [ 0.,  0.,  0., ...,  0.,  0.,  0.],
         ..., 
         [ 0.,  0.,  0., ...,  0.,  0.,  0.],
         [ 0.,  0.,  0., ...,  0.,  0.,  0.],
         [ 0.,  0.,  0., ...,  0.,  0.,  0.]],
 
        [[ 0.,  0.,  0., ...,  0.,  0.,  0.],
         [ 0.,  0.,  0., ...,  0.,  0.,  0.],
         [ 0.,  0.,  0., ...,  0.,  0.,  0.],
         ..., 
         [ 0.,  0.,  0., ...,  0.,  0.,  0.],
         [ 0.,  0.,  0., ...,  0.,  0.,  0.],
         [ 0.,  0.,  0., ...,  0.,  0.,  0.]],
 
        [[ 0.,  0.,  0., ...,  0.,  0.,  0.],
         [ 1.,  0.,  0., ...,  0.,  0.,  0.],
         [ 0.,  0.,  0., ...,  0.,  0.,  0.],
         ..., 
         [ 0.,  0.,  0., ...,  0.,  0.,  0.],
         [ 0.,  0.,  0., ...,  0.,  0.,  0.],
         [ 0.,  0.,  0., ...,  0.,  0.,  0.]],
 
        ..., 
        [[ 0.,  0.,  0., ...,  0.,  0.,  0.],
         [ 0.,  0.,  0., ...,  0.,  0.,  0.],
         [ 1

In [54]:
def create_char_cnn_model(num_chars, max_sequence_len, num_labels):
    char_input = Input(shape=(max_sequence_len, num_chars), name='char_cnn_input')
    
    conv_1x = Conv1D(128, 6, activation='relu', padding='valid')(char_input)
    max_pool_1x = MaxPooling1D(4)(conv_1x)
    conv_2x = Conv1D(256, 6, activation='relu', padding='valid')(max_pool_1x)
    max_pool_2x = MaxPooling1D(4)(conv_2x)

    flatten = Flatten()(max_pool_2x)
    dense = Dense(128, activation='relu')(flatten)
    preds = Dense(num_labels, activation='softmax', name='char_cnn_predictions')(dense)

    model = Model(char_input, preds)
    model.compile(loss='sparse_categorical_crossentropy',
                  optimizer='rmsprop',
                  metrics=['acc'])
    return model

char_cnn_model = create_char_cnn_model(len(char_to_idx), max_sequence_len, len(emojis))
char_cnn_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
char_cnn_input (InputLayer)  (None, 139, 96)           0         
_________________________________________________________________
conv1d_33 (Conv1D)           (None, 134, 128)          73856     
_________________________________________________________________
max_pooling1d_27 (MaxPooling (None, 33, 128)           0         
_________________________________________________________________
conv1d_34 (Conv1D)           (None, 28, 256)           196864    
_________________________________________________________________
max_pooling1d_28 (MaxPooling (None, 7, 256)            0         
_________________________________________________________________
flatten_4 (Flatten)          (None, 1792)              0         
_________________________________________________________________
dense_7 (Dense)              (None, 128)               229504    
__________

In [13]:
early = keras.callbacks.EarlyStopping(monitor='loss',
                              min_delta=0.03,
                              patience=2,
                              verbose=0, mode='auto')

BATCH_SIZE = 512
char_cnn_model.fit_generator(
    data_generator(train_tweets, batch_size=BATCH_SIZE),
    epochs=20,
    steps_per_epoch=len(train_tweets) / BATCH_SIZE,
    verbose=2,
    callbacks=[early]
)

Epoch 1/20
73s - loss: 3.6955 - acc: 0.2050
Epoch 2/20
65s - loss: 3.3265 - acc: 0.2616
Epoch 3/20
65s - loss: 3.1551 - acc: 0.2957
Epoch 4/20
65s - loss: 3.0320 - acc: 0.3192
Epoch 5/20
65s - loss: 2.9288 - acc: 0.3405
Epoch 6/20
65s - loss: 2.8463 - acc: 0.3574
Epoch 7/20
65s - loss: 2.7718 - acc: 0.3721
Epoch 8/20
65s - loss: 2.7119 - acc: 0.3843
Epoch 9/20
65s - loss: 2.6648 - acc: 0.3941
Epoch 10/20
65s - loss: 2.6333 - acc: 0.3999
Epoch 11/20
65s - loss: 2.5973 - acc: 0.4064
Epoch 12/20
65s - loss: 2.5609 - acc: 0.4137
Epoch 13/20
65s - loss: 2.5355 - acc: 0.4183
Epoch 14/20
64s - loss: 2.5126 - acc: 0.4218
Epoch 15/20
65s - loss: 2.4913 - acc: 0.4261
Epoch 16/20
65s - loss: 2.4705 - acc: 0.4303
Epoch 17/20
65s - loss: 2.4618 - acc: 0.4305
Epoch 18/20
65s - loss: 2.4440 - acc: 0.4337
Epoch 19/20
65s - loss: 2.4231 - acc: 0.4375
Epoch 20/20
65s - loss: 2.4104 - acc: 0.4395


In [17]:
char_cnn_model.evaluate_generator(
    data_generator(test_tweets, batch_size=BATCH_SIZE),
    steps=len(test_tweets) / BATCH_SIZE
)

[3.1370692166729248, 0.3634086277173913]

In [13]:
with open('zoo/07/emoji_chars.json', 'w') as fout:
    json.dump({
        'emojis': ''.join(emojis),
        'char_to_idx': char_to_idx,
        'max_sequence_len': max_sequence_len,
    }, fout)
char_cnn_model.save('zoo/07/char_cnn_model.h5')
char_cnn_model.save_weights('zoo/07/char_cnn_model_weights.h5')

In [18]:
pd.options.display.max_colwidth = 128
inspect_tweets = test_tweets.sample(100)
predicted = char_cnn_model.predict_generator(data_generator(inspect_tweets, batch_size=None), steps=1)
show = pd.DataFrame({
    'text': inspect_tweets['text'],
    'true': inspect_tweets['emoji'],
    'pred': [emojis[np.argmax(x)] for x in predicted],
})
show = show[['text', 'true', 'pred']]
show.head(10)

,text,true,pred
96993,@lovereina_: I really wanna be caked up rn,🙄,😩
21238,@PeopIe: Awesome friendship,😀,😀
637878,"colourbee: Chibi Star and Marco, my first Star vs the forces of evil fanart yay :3",📷,😂
420818,"@howluans: I love you so much, you make me happy just to smile. I hope one day you notice me here @camilacabello97",❤,❤
223513,Go to hell,😈,😂
759685,My header poppin,😍,😂
282387,Cesar is a fucking mongol,😂,😂
534106,@ComplexSports: OMFG,💀,💀
249189,Let's have a chat now O <== Here i am,⏭,👉
534534,There's a house centipede in my closet and since Ryan isn't here I have to kill it....,🙃,🙃


In [55]:
from keras.layers import Input, Conv1D, MaxPooling1D, Flatten, Dense, Dropout, Merge, LSTM
from keras.models import Model
from keras.layers.merge import Concatenate

def create_char_cnn_model2(num_chars, max_sequence_len, num_labels, drop_out=0.25):
    char_input = Input(shape=(max_sequence_len, num_chars), name='char_cnn_input')
    
    layers = []
    for window in (4, 5, 6):
        conv_1x = Conv1D(128, window, activation='relu', padding='valid')(char_input)
        max_pool_1x = MaxPooling1D(4)(conv_1x)
        dropout_1x = Dropout(drop_out)(max_pool_1x)
        conv_2x = Conv1D(256, window, activation='relu', padding='valid')(dropout_1x)
        max_pool_2x = MaxPooling1D(4)(conv_2x)
        dropout_2x = Dropout(drop_out)(max_pool_2x)
        layers.append(dropout_2x)

    merged = Concatenate(axis=1)(layers)

    dropout = Dropout(drop_out)(merged)
    
    flatten = Flatten()(dropout)
    dense = Dense(128, activation='relu')(flatten)
    preds = Dense(num_labels, activation='softmax', name='char_cnn_predictions')(dense)

    model = Model(char_input, preds)
    model.compile(loss='sparse_categorical_crossentropy',
                  optimizer='rmsprop',
                  metrics=['acc'])
    return model

char_cnn_model2 = create_char_cnn_model2(len(char_to_idx), max_sequence_len, len(emojis))
char_cnn_model2.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
char_cnn_input (InputLayer)      (None, 139, 96)       0                                            
____________________________________________________________________________________________________
conv1d_35 (Conv1D)               (None, 136, 128)      49280                                        
____________________________________________________________________________________________________
conv1d_37 (Conv1D)               (None, 135, 128)      61568                                        
____________________________________________________________________________________________________
conv1d_39 (Conv1D)               (None, 134, 128)      73856                                        
___________________________________________________________________________________________

In [20]:
BATCH_SIZE = 2048
char_cnn_model2.fit_generator(
    data_generator(train_tweets, batch_size=BATCH_SIZE),
    epochs=30,
    steps_per_epoch=len(train_tweets) / BATCH_SIZE,
    verbose=2,
    callbacks=[early]
)

Epoch 1/30
59s - loss: 3.8784 - acc: 0.1843
Epoch 2/30
58s - loss: 3.5497 - acc: 0.2219
Epoch 3/30
58s - loss: 3.3917 - acc: 0.2501
Epoch 4/30
58s - loss: 3.2934 - acc: 0.2690
Epoch 5/30
58s - loss: 3.2190 - acc: 0.2835
Epoch 6/30
58s - loss: 3.1664 - acc: 0.2923
Epoch 7/30
58s - loss: 3.1136 - acc: 0.3020
Epoch 8/30
58s - loss: 3.0729 - acc: 0.3096
Epoch 9/30
58s - loss: 3.0430 - acc: 0.3154
Epoch 10/30
58s - loss: 3.0064 - acc: 0.3220
Epoch 11/30
58s - loss: 2.9784 - acc: 0.3268
Epoch 12/30
58s - loss: 2.9557 - acc: 0.3317
Epoch 13/30
58s - loss: 2.9331 - acc: 0.3365
Epoch 14/30
58s - loss: 2.9123 - acc: 0.3401
Epoch 15/30
58s - loss: 2.8850 - acc: 0.3453
Epoch 16/30
58s - loss: 2.8741 - acc: 0.3477
Epoch 17/30
58s - loss: 2.8603 - acc: 0.3491
Epoch 18/30
58s - loss: 2.8436 - acc: 0.3536
Epoch 19/30
58s - loss: 2.8279 - acc: 0.3562
Epoch 20/30
58s - loss: 2.8100 - acc: 0.3599
Epoch 21/30
58s - loss: 2.8045 - acc: 0.3600
Epoch 22/30
58s - loss: 2.7895 - acc: 0.3643
Epoch 23/30
58s - l

In [21]:
char_cnn_model2.evaluate_generator(
    data_generator(test_tweets, batch_size=BATCH_SIZE),
    steps=len(test_tweets) / BATCH_SIZE
)

[2.8094391482216969, 0.37586495535714287]

In [ ]:
"75s - loss: 2.3855 - acc: 0.4368\n[2.8089022636413574, 0.38840296648550726]"

## Featurizing and preparing our data

Just like we did when computing word embeddings, we want to featurize our data so we can classify it effectively.

In [22]:
VOCAB_SIZE = 50000
tokenizer = Tokenizer(num_words=VOCAB_SIZE)
tokenizer.fit_on_texts(tweets['text'])

In [23]:
training_tokens = tokenizer.texts_to_sequences(train_tweets['text'])
test_tokens = tokenizer.texts_to_sequences(test_tweets['text'])
max_num_tokens = max(len(x) for x in chain(training_tokens, test_tokens))
training_tokens = pad_sequences(training_tokens, maxlen=max_num_tokens)
test_tokens = pad_sequences(test_tokens, maxlen=max_num_tokens)

In [24]:
training_labels = np.asarray([emoji_to_idx[em] for em in train_tweets['emoji']])
test_labels = np.asarray([emoji_to_idx[em] for em in test_tweets['emoji']])

In [67]:
def load_weights(tokenizer):
    model = Word2Vec.load('data/twitter_w2v.model')
    w2v = np.zeros((tokenizer.num_words, w2v_model.syn0.shape[1]))
    for k, v in tokenizer.word_index.items():
        if v >= tokenizer.num_words:
            continue
        if k in w2v_model:
            w2v[v] = w2v_model[k]
    return w2v

In [74]:
# This may take a while to load

#w2v = load_weights(tokenizer)

#model = Word2Vec.load('data/twitter_w2v.model')
w2v = np.zeros((tokenizer.num_words, model.wv.syn0.shape[1]))
found = 0
for k, v in tokenizer.word_index.items():
    if v >= tokenizer.num_words:
        continue
    if k in model:
        w2v[v] = model[k]
        found += 1
found, tokenizer.num_words


(49874, 50000)

# World Level CNN

As with our previous task, we can try using more powerful models to classify our text.  In this case, the limited training data and text size limit their effectiveness.

In [77]:
def create_cnn_model(vocab_size, embedding_size=None, embedding_weights=None, drop_out=0.2):
    message = Input(shape=(max_num_tokens,), dtype='int32', name='cnn_input')
    
    
    # The convolution layer in keras does not support masking, so we just allow
    # the embedding layer to learn an explicit value.
    embedding = Embedding(mask_zero=False, input_dim=vocab_size, 
                          output_dim=embedding_weights.shape[1], 
                          weights=[embedding_weights],
                          trainable=True,
                          name='cnn_embedding')(message)
    
    global_pools = []
    for window in 2, 3:
        conv_1x = Conv1D(128, window, activation='relu', padding='valid')(embedding)
        max_pool_1x = MaxPooling1D(2)(conv_1x)
        conv_2x = Conv1D(256, window, activation='relu', padding='valid')(max_pool_1x)
        max_pool_2x = MaxPooling1D(2)(conv_2x)
        conv_3x = Conv1D(256, window, activation='relu', padding='valid')(max_pool_2x)

        global_pools.append(GlobalMaxPooling1D()(conv_3x))

    merged = Concatenate(axis=1)(global_pools)
    fc1 = Dense(units=128, activation='elu')(merged)
    preds = Dense(units=len(emojis), activation='softmax', name='cnn_predictions')(fc1)
    model = Model(
        inputs=[message],
        outputs=[preds],
    )
    model.compile(loss='sparse_categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
    return model

cnn_model = create_cnn_model(VOCAB_SIZE, embedding_weights=w2v)
cnn_model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
cnn_input (InputLayer)           (None, 54)            0                                            
____________________________________________________________________________________________________
cnn_embedding (Embedding)        (None, 54, 100)       5000000                                      
____________________________________________________________________________________________________
conv1d_53 (Conv1D)               (None, 53, 128)       25728                                        
____________________________________________________________________________________________________
conv1d_56 (Conv1D)               (None, 52, 128)       38528                                        
___________________________________________________________________________________________

In [79]:
cnn_model.fit(training_tokens, training_labels, epochs=5)

Epoch 1/5
634695/634695 [==============================] - 112s - loss: 3.2709 - acc: 0.3009   
Epoch 2/5
634695/634695 [==============================] - 113s - loss: 3.2831 - acc: 0.3012   
Epoch 3/5
634695/634695 [==============================] - 112s - loss: 3.2785 - acc: 0.3023   
Epoch 4/5
634695/634695 [==============================] - 112s - loss: 3.2929 - acc: 0.3006   
Epoch 5/5
634695/634695 [==============================] - 112s - loss: 3.2968 - acc: 0.2990   


In [57]:
def create_lstm_model(vocab_size, embedding_size=None, embedding_weights=None):
    message = Input(shape=(None,), dtype='int32', name='lstm_input')
    embedding = Embedding(mask_zero=False, input_dim=vocab_size, 
                          output_dim=embedding_weights.shape[1], 
                          weights=[embedding_weights],
                          trainable=True,
                          name='lstm_embedding')(message)

    lstm_1 = LSTM(units=128, return_sequences=False)(embedding)
    preds = Dense(units=len(emojis), activation='softmax', name='lstm_predictions')(lstm_1)
    
    model = Model(
        inputs=[message],
        outputs=[preds],
    )
    model.compile(loss='sparse_categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
    return model

In [53]:
lstm_model = create_lstm_model(VOCAB_SIZE, embedding_weights=w2v)
lstm_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_input (InputLayer)      (None, None)              0         
_________________________________________________________________
lstm_embedding (Embedding)   (None, None, 300)         15000000  
_________________________________________________________________
lstm_3 (LSTM)                (None, 128)               219648    
_________________________________________________________________
lstm_predictions (Dense)     (None, 121)               15609     
Total params: 15,235,257
Trainable params: 15,235,257
Non-trainable params: 0
_________________________________________________________________


In [39]:
lstm_model.fit(training_tokens, training_labels, epochs=12, batch_size=1024, callbacks=[early])

Epoch 1/12
634695/634695 [==============================] - 40s - loss: 3.3607 - acc: 0.2660    
Epoch 2/12
634695/634695 [==============================] - 40s - loss: 2.8690 - acc: 0.3510    
Epoch 3/12
634695/634695 [==============================] - 40s - loss: 2.6476 - acc: 0.3937    
Epoch 4/12
634695/634695 [==============================] - 40s - loss: 2.4931 - acc: 0.4225    
Epoch 5/12
634695/634695 [==============================] - 40s - loss: 2.3727 - acc: 0.4445    
Epoch 6/12
634695/634695 [==============================] - 40s - loss: 2.2696 - acc: 0.4645    
Epoch 7/12
634695/634695 [==============================] - 40s - loss: 2.1778 - acc: 0.4822    
Epoch 8/12
634695/634695 [==============================] - 40s - loss: 2.0931 - acc: 0.4989    
Epoch 9/12
634695/634695 [==============================] - 40s - loss: 2.0135 - acc: 0.5154    
Epoch 10/12
634695/634695 [==============================] - 40s - loss: 1.9375 - acc: 0.5314    
Epoch 11/12
634695/634695 [==

In [40]:
lstm_model.evaluate(test_tokens, test_labels)

70522/70522 [==============================] - 24s    


[2.8785435600453315, 0.4029948101352433]

## Comparing our models

Let's compare the predictions from our models on a sample of our data.

In [41]:
test_char_vectors, _ = next(data_generator(test_tweets, None)) 

In [42]:
predictions = {
    label: [emojis[np.argmax(x)] for x in pred]
    for label, pred in (
        ('lstm', lstm_model.predict(test_tokens[:100])),
        ('char_cnn', char_cnn_model.predict(test_char_vectors[:100])),
        ('cnn', cnn_model.predict(test_tokens[:100])),
    )
}

In [43]:
# Make a dataframe just for test data
pd.options.display.max_colwidth = 128
test_df = test_tweets[:100].reset_index()
eval_df = pd.DataFrame({
    'content': test_df['text'],
    'true': test_df['emoji'],
    **predictions
})
eval_df[['content', 'true', 'char_cnn', 'cnn', 'lstm']].head(25)

,content,true,char_cnn,cnn,lstm
0,@Gurmeetramrahim @RedFMIndia @rjraunac #8DaysToLionHeart Great,👏,👍,👏,😘
1,@suchsmallgods I can't wait to show him these tweets,😈,😂,❤,😭
2,@Captain_RedWolf I have like 20 set lol WAYYYYYY ahead of you,😂,😂,😂,😂
3,".@OtherkinOK were just at @EPfestival, what a set! Next stop is @whelanslive on Friday 11th November 2016.",👌,💪,❤,😊
4,@jochendria: KathNiel with GForce Jorge. #PushAwardsKathNiels,💙,💙,💙,💙
5,Okay good,😂,😂,❤,😂
6,"""Distraught means to be upset"" ""So that means confused right?"" -@ReevesDakota",😂,😂,❤,😭
7,@JennLiri babe wtf call bck I'm tryna listen to this ring tone,🙄,🙄,©,😩
8,"does Jen want to be friends? we can so be friends. love you, girl. #BachelorInParadise",❤,😢,❤,❤
9,@amwalker38: Go Follow these hot accounts @the1stMe420 @DanaDeelish @So_deelish @aka_teemoney38 @CamPromoXXX @SexyLThings @l...,👇,👑,🔥,✨


## Qualitative Evaluation

We can examine some of our error cases by hand.  Often, the models tend to agree when they make mistakes, and that the mistakes aren't unreasonable: this task would be challenging even for a human.

In [44]:
eval_df[eval_df['lstm'] != eval_df['true']].head(10)

,char_cnn,cnn,content,lstm,true
0,👍,👏,@Gurmeetramrahim @RedFMIndia @rjraunac #8DaysToLionHeart Great,😘,👏
1,😂,❤,@suchsmallgods I can't wait to show him these tweets,😭,😈
3,💪,❤,".@OtherkinOK were just at @EPfestival, what a set! Next stop is @whelanslive on Friday 11th November 2016.",😊,👌
6,😂,❤,"""Distraught means to be upset"" ""So that means confused right?"" -@ReevesDakota",😭,😂
7,🙄,©,@JennLiri babe wtf call bck I'm tryna listen to this ring tone,😩,🙄
9,👑,🔥,@amwalker38: Go Follow these hot accounts @the1stMe420 @DanaDeelish @So_deelish @aka_teemoney38 @CamPromoXXX @SexyLThings @l...,✨,👇
10,🙃,😂,@gspisak: I always made fun of the parents that show up 30+ mins early to pick up their kids today thats me At least I got a...,😂,🙈
12,😂,😂,@kayleeburt77 can I have your number? I seem to have lost mine.,🤔,😂
15,😂,😂,@RoxeteraRibbons Same and I have to figure to prove it,😊,😂
17,❤,😘,@VixenMusicLabel - Peace & Love,❤,✌


In [61]:
def combined_data_generator(tweets, tokens, batch_size):
    tweets = tweets.reset_index()
    while True:
        batch_idx = random.sample(range(len(tweets)), batch_size)
        tweet_batch = tweets.iloc[batch_idx]
        token_batch = tokens[batch_idx]
        char_vec = np.zeros((batch_size, max_sequence_len, len(chars)))
        token_vec = np.zeros((batch_size, max_num_tokens))
        y = np.zeros((batch_size,))
        for row_idx, (token_row, (_, tweet_row)) in enumerate(zip(token_batch, tweet_batch.iterrows())):
            y[row_idx] = emoji_to_idx[tweet_row['emoji']]
            for ch_idx, ch in enumerate(tweet_row['text']):
                char_vec[row_idx, ch_idx, char_to_idx[ch]] = 1
            token_vec[row_idx, :] = token_row
        yield {'char_cnn_input': char_vec, 'cnn_input': token_vec, 'lstm_input': token_vec}, y

d, y = next(combined_data_generator(train_tweets, training_tokens, 5))
d['lstm_input'].shape

(5, 54)

In [59]:
def prediction_layer(model):
    layers = [layer for layer in model.layers if layer.name.endswith('_predictions')]
    return layers[0].output

def create_ensemble(*models):
    inputs = [model.input for model in models]
    predictions = [prediction_layer(model) for model in models]
    merged = Average()(predictions)
    model = Model(
        inputs=inputs,
        outputs=[merged],
    )
    model.compile(loss='sparse_categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
    return model


ensemble = create_ensemble(char_cnn_model2, cnn_model, lstm_model)
ensemble.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
char_cnn_input (InputLayer)      (None, 139, 96)       0                                            
____________________________________________________________________________________________________
conv1d_35 (Conv1D)               (None, 136, 128)      49280                                        
____________________________________________________________________________________________________
conv1d_37 (Conv1D)               (None, 135, 128)      61568                                        
____________________________________________________________________________________________________
conv1d_39 (Conv1D)               (None, 134, 128)      73856                                        
___________________________________________________________________________________________

In [62]:
BATCH_SIZE = 512
ensemble.fit_generator(
    combined_data_generator(train_tweets, training_tokens, BATCH_SIZE),
    epochs=20,
    steps_per_epoch=len(train_tweets) / BATCH_SIZE,
    verbose=2,
    callbacks=[early]
)

Epoch 1/20
162s - loss: 3.2924 - acc: 0.2773
Epoch 2/20
243s - loss: 2.8232 - acc: 0.3604
Epoch 3/20
269s - loss: 2.6129 - acc: 0.4015
Epoch 4/20
260s - loss: 2.4619 - acc: 0.4321
Epoch 5/20
437s - loss: 2.3398 - acc: 0.4576
Epoch 6/20
498s - loss: 2.2380 - acc: 0.4777
Epoch 7/20
576s - loss: 2.1480 - acc: 0.4968
Epoch 8/20
193s - loss: 2.0651 - acc: 0.5144
Epoch 9/20
351s - loss: 1.9845 - acc: 0.5327
Epoch 10/20
413s - loss: 1.9129 - acc: 0.5482
Epoch 11/20
757s - loss: 1.8547 - acc: 0.5607
Epoch 12/20
768s - loss: 1.7917 - acc: 0.5757
Epoch 13/20
786s - loss: 1.7301 - acc: 0.5886
Epoch 14/20
784s - loss: 1.6808 - acc: 0.5997
Epoch 15/20
780s - loss: 1.6333 - acc: 0.6101
Epoch 16/20
761s - loss: 1.5852 - acc: 0.6204
Epoch 17/20
751s - loss: 1.5440 - acc: 0.6284
Epoch 18/20


KeyboardInterrupt: 

In [65]:
ensemble.evaluate_generator(
    combined_data_generator(test_tweets, test_tokens, BATCH_SIZE),
    steps=len(test_tweets) / BATCH_SIZE
)

[2.9514826242474541, 0.38115658967391303]

In [80]:
len(train_tweets)

634695